# Calculate SSIM/PSNR values for ROI

## Generate Masks
Clone https://github.com/jasonccai/HeadCTSegmentation and download the model weights. 
Copy z_unet.py and weights.hdf5 into this folder

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import sys
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import pandas as pd
from scipy.ndimage import binary_fill_holes
from scipy.stats import shapiro


In [ ]:
#Load segmentation model
from z_unet import unet

U_Net = unet(nb_classes=17, savefolder="./", flag="a")
U_Net.load_weights("./weights.hdf5")

In [ ]:
if not os.path.exists("./Masks/"):
    os.mkdir("./Masks/")
    

### RSNA test split

In [ ]:
test_list = pd.read_csv("./test_data.csv")["filename"].values

In [ ]:
#test segmentation model
for filename in test_list[0:3]:
    filename = filename.replace(".dcm", ".npy")
    img = np.load("./Data/4096/" + filename)*4095 - 1024

    pred = U_Net.predict(img[None, :, :, None])
    pred_re = pred.reshape(512, 512, 17)
    
    plt.figure()
    plt.imshow(img.squeeze(), cmap='gray', vmin=0, vmax=80)
    plt.imshow(1-pred_re[:, :, 0], alpha=0.4, cmap='Reds', vmin=0, vmax=1)
    segmentation = img.copy()
    segmentation[pred_re[:, :, 0]>0.3] = 0 
    segmentation[pred_re[:, :, 0]<=0.3] = 1
    mask = binary_fill_holes(segmentation)

    plt.figure()
    plt.imshow(mask, cmap='gray')
    #np.save(f"./Masks/mask_{filename}", segmentation)

In [ ]:
#loop trough entire testset and save ROI masks for each file 
for i, filename in enumerate(test_list):
    sys.stdout.write('\r'+f'{i}/{len(test_list)}')    
    filename = filename.replace(".dcm", ".npy")
    img = np.load("./Data/4096/" + filename) - 1024

    pred = U_Net.predict(img[None, :, :, None], verbose=False)
    pred_re = pred.reshape(512, 512, 17)
    
    segmentation = img.copy()
    segmentation[pred_re[:, :, 0]>0.3] = 0 
    segmentation[pred_re[:, :, 0]<=0.3] = 1
    
    np.save(f"./Masks/mask_{filename}", segmentation)
    

### Repeat for CQ500 dataset

In [ ]:
test_list = os.listdir("/data-pool/data_no_backup/ga63cun/CQ500/4096/")
#test segmentation model
for filename in test_list[1440:1443]:
    filename = filename.replace(".dcm", ".npy")
    print(filename)
    img = np.load("./Data/CQ500/4096/" + filename)*4095 - 1024

    pred = U_Net.predict(img[None, :, :, None])
    pred_re = pred.reshape(512, 512, 17)
    
    plt.figure()
    plt.imshow(img.squeeze(), cmap='gray', vmin=0, vmax=80)
    plt.imshow(1-pred_re[:, :, 0], alpha=0.4, cmap='Reds', vmin=0, vmax=1)
    segmentation = img.copy()
    segmentation[pred_re[:, :, 0]>0.3] = 0 
    segmentation[pred_re[:, :, 0]<=0.3] = 1
    mask = binary_fill_holes(segmentation)

    plt.figure()
    plt.imshow(mask, cmap='gray')


In [ ]:
#loop trough entire testset and save ROI masks for each file 
for i, filename in enumerate(test_list[::-1]):
    sys.stdout.write('\r'+f'{i}/{len(test_list)}')    
    filename = filename.replace(".dcm", ".npy")
    img = np.load("./Data/CQ500/4096/" + filename)*4095 - 1024

    pred = U_Net.predict(img[None, :, :, None], verbose=False)
    pred_re = pred.reshape(512, 512, 17)
    
    segmentation = img.copy()
    segmentation[pred_re[:, :, 0]>0.3] = 0 
    segmentation[pred_re[:, :, 0]<=0.3] = 1
    
    np.save(f"./Masks/mask_{filename}", segmentation)

In [ ]:
del U_Net

## Calculate PSNR/SSIM

### RSNA test split

In [ ]:
from skimage.util.arraycrop import crop

def ROI_ssim(img1, img2, mask, data_range=1):
    
    """
    Calculate the SSIM of two images for a region of interest (ROI). 
    
    Parameters:
    img1, img2: array
    The two images to be compared
    mask: array
    binary mask of ROI
    data_range: int/float
    data range of input data
    
    returns:
    ssim_roi: float
    ssim in region of interest
    """
    ssim_value, ssim_map = ssim(img1, img2, data_range=1, full=True)
    
    #crop to ignore edge effects. Only relevant if mask = 1 for all pixels. 
 
    pad = (7 - 1) // 2
    ssim_map, mask = crop(ssim_map, pad), crop(mask, pad)
    
    ssim_roi = np.sum(ssim_map*mask)/np.sum(mask)
    return ssim_roi


def ROI_psnr(img1, img2, mask, data_range=1):
    
    """
    Calculate the PSNR of two images for a region of interest (ROI).
    
    Parameters:
    img1, img2: array
    The two images to be compared
    mask: array
    binary mask of ROI
    data_range: int/float
    data range of input data
    
    
    returns:
    psnr_roi: float
    psnr in region of interest
    """
    err_roi = np.sum((img1*mask - img2*mask) ** 2, dtype=np.float64)/np.sum(mask)
    psnr_roi = 10 * np.log10((data_range ** 2) / err_roi)
    return psnr_roi


## check if everything works as expected

In [ ]:
test_list = pd.read_csv("./test_data.csv")["filename"].values

for i, filename in enumerate(test_list[0:3]):
    #sys.stdout.write('\r'+f'{i}/{len(test_list)}')    
    filename = filename.replace(".dcm", ".npy")
    gt = np.load("./Data/4096/" + filename)/4095
    sparse = np.load("./Data/256/" + filename)/4095
    mask = np.load("./Masks/mask_" + filename)
    mask = binary_fill_holes(mask)
    
    #uncomment to test if ROI_ functions
    #mask = np.ones_like(mask)

    #skip iterations where mask is zero
    if 1 not in mask:
        continue
    
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
    ax1.imshow(gt, cmap='gray', vmin=(0+1024)/4095, vmax=(80+1024)/4095)
    ax2.imshow(sparse, cmap='gray', vmin=(0+1024)/4095, vmax=(80+1024)/4095)
    ax3.imshow(mask, cmap='gray', vmin=0, vmax=1)
    
    psnr_roi = ROI_psnr(gt, sparse, mask)
    ssim_roi = ROI_ssim(gt, sparse, mask)
    
    psnr_full = psnr(gt, sparse, data_range=1)
    ssim_full = ssim(gt, sparse, data_range=1)
    
    print(psnr_roi, psnr_full, ssim_roi, ssim_full)

## loop trough testset

In [ ]:
from skimage.restoration import denoise_tv_chambolle
from models import *

test_list = pd.read_csv("./Data/test_data.csv")["filename"].values

unet = U_Net()


weights = {2048:0.001, 1024:0.002, 512:0.009, 256:0.045, 128:0.153, 64:0.570}

results = pd.DataFrame()
angles = [2048, 1024, 512, 256, 128, 64]
#angles = [2048]
for angle in angles:
    print('\n', angle)
    
    #load U-Net for specific angle
    unet = U_Net()
    unet.load_weights(f'./model_weights/U-Net/{angle}/model_75.h5')
    ssim_list_sparse = []
    psnr_list_sparse = []

    ssim_list_U_Net = []
    psnr_list_U_Net = []
    
    ssim_list_TV = []
    psnr_list_TV = []
    
    ssim_full_list_U_Net = []

    for i, filename in enumerate(test_list):
        sys.stdout.write('\r'+f'{i}/{len(test_list)}')    
        filename = filename.replace(".dcm", ".npy")
        gt = np.load("./Data/4096/" + filename)/4095
        sparse = np.load(f"./Data/{angle}/" + filename)/4095
        mask = np.load("./Masks/mask_" + filename)
        mask = binary_fill_holes(mask)

        #skip iterations where mask is zero
        if 1 not in mask:
            continue
        
        psnr_list_sparse.append(ROI_psnr(gt, sparse, mask))
        ssim_list_sparse.append(ROI_ssim(gt, sparse, mask))
        
        #calculate psnr + ssim for predicted data
        img_U_Net = unet.predict(sparse[np.newaxis, :, :, np.newaxis], verbose=False)
        img_U_Net = np.squeeze(img_U_Net)
        psnr_list_U_Net.append(ROI_psnr(gt, img_U_Net, mask))
        ssim_list_U_Net.append(ROI_ssim(gt, img_U_Net, mask))
        ssim_full_list_U_Net.append(ssim(gt, img_U_Net, data_range=1))
        
        #calculate psnr + ssim for tv data
        img_tv = denoise_tv_chambolle(sparse, weight=weights[angle])
        psnr_list_TV.append(ROI_psnr(gt, img_tv, mask))
        ssim_list_TV.append(ROI_ssim(gt, img_tv, mask))
    
    results[f'psnr_TV_{angle}'] = psnr_list_TV
    results[f'ssim_TV_{angle}'] = ssim_list_TV
    
    results[f'psnr_sparse_{angle}'] = psnr_list_sparse
    results[f'ssim_sparse_{angle}'] = ssim_list_sparse

    results[f'psnr_U_Net_{angle}'] = psnr_list_U_Net
    results[f'ssim_U_Net_{angle}'] = ssim_list_U_Net
    results[f'ssim_full_U_Net_{angle}'] = ssim_full_list_U_Net

results.to_csv('./ROI_psnr_ssim_RSNA.csv')
results.head()

### CQ500

In [ ]:
from skimage.restoration import denoise_tv_chambolle
from models import *

test_list = os.listdir("/data-pool/data_no_backup/ga63cun/CQ500/4096/")
test_list = [element for element in test_list if ".npy" in element]

unet = U_Net()


weights = {2048:0.001, 1024:0.002, 512:0.009, 256:0.045, 128:0.153, 64:0.570}

results = pd.DataFrame()
angles = [2048, 1024, 512, 256, 128, 64]
#angles = [2048]
for angle in angles:
    print('\n', angle)
    
    #load U-Net for specific angle
    unet = U_Net()
    unet.load_weights(f'./model_weights/U-Net/{angle}/model_75.h5')
    ssim_list_sparse = []
    psnr_list_sparse = []

    ssim_list_U_Net = []
    psnr_list_U_Net = []
    
    ssim_list_TV = []
    psnr_list_TV = []
    
    ssim_full_list_U_Net = []

    for i, filename in enumerate(test_list):
        sys.stdout.write('\r'+f'{i}/{len(test_list)}')    
        filename = filename.replace(".dcm", ".npy")
        gt = np.load("./CQ500/Data/4096/" + filename)/4095
        sparse = np.load(f"./CQ500/Data/{angle}/" + filename)/4095
        mask = np.load("./Masks/mask_" + filename)
        mask = binary_fill_holes(mask)

        #skip iterations where mask is zero
        if 1 not in mask:
            continue
        
        psnr_list_sparse.append(ROI_psnr(gt, sparse, mask))
        ssim_list_sparse.append(ROI_ssim(gt, sparse, mask))
        
        #calculate psnr + ssim for predicted data
        img_U_Net = unet.predict(sparse[np.newaxis, :, :, np.newaxis], verbose=False)
        img_U_Net = np.squeeze(img_U_Net)
        psnr_list_U_Net.append(ROI_psnr(gt, img_U_Net, mask))
        ssim_list_U_Net.append(ROI_ssim(gt, img_U_Net, mask))
        ssim_full_list_U_Net.append(ssim(gt, img_U_Net, data_range=1))
        
        #calculate psnr + ssim for tv dataf
        img_tv = denoise_tv_chambolle(sparse, weight=weights[angle])
        psnr_list_TV.append(ROI_psnr(gt, img_tv, mask))
        ssim_list_TV.append(ROI_ssim(gt, img_tv, mask))
    
    results[f'psnr_TV_{angle}'] = psnr_list_TV
    results[f'ssim_TV_{angle}'] = ssim_list_TV
    
    results[f'psnr_sparse_{angle}'] = psnr_list_sparse
    results[f'ssim_sparse_{angle}'] = ssim_list_sparse

    results[f'psnr_U_Net_{angle}'] = psnr_list_U_Net
    results[f'ssim_U_Net_{angle}'] = ssim_list_U_Net
    results[f'ssim_full_U_Net_{angle}'] = ssim_full_list_U_Net

    results.to_csv('./ROI_psnr_ssim_CQ500.csv')
results.head()

## Calculate mean values and CIs

In [ ]:
import scipy.stats as st

def wilco_func(x, y, alternative='two-sided', method='auto'):
    out = st.wilcoxon(x, y, alternative=alternative, method=method)
    return round(out[0], 5), round(out[1], 5)


### RSNA test split

In [ ]:
df_mean = pd.DataFrame()
df_rsna = pd.read_csv("./ROI_psnr_ssim_RSNA.csv")
df_rsna = df_rsna.replace([np.inf, -np.inf], np.nan).dropna()
angles = [2048, 1024, 512, 256, 128, 64]

for angle in angles:
    column = []
    
    for typ in ["ssim", "psnr"]:
        for proc in ["sparse", "U_Net", "TV"]:

            print(angle, proc)
            metric_list = df_rsna[f'{typ}_{proc}_{angle}']
            res = st.bootstrap((metric_list,), np.mean, n_resamples=1000, confidence_level=0.95)
            metric_interval = res.confidence_interval
            
            column.append(f'{round(metric_list.mean(), 3)}')
            column.append(f'({round(metric_interval[0], 3)}-{round(metric_interval[1], 3)})')
            
    df_mean[f"{angle}"] = column

            #check for normal distribution
            #print(f"{typ}", shapiro(metric_list))
df_mean.to_csv("./table_ROI_SSIM_PSNR_raw.csv")

In [ ]:
#check if distributions are significantly different
angles = [2048, 1024, 512, 256, 128, 64]
typ="ssim"
for angle in angles:
    stat_sparse_tv = wilco_func(df_rsna[f'{typ}_sparse_{angle}'], df_rsna[f'{typ}_TV_{angle}'])
    stat_sparse_U_Net = wilco_func(df_rsna[f'{typ}_sparse_{angle}'], df_rsna[f'{typ}_U_Net_{angle}'])
    stat_TV_U_Net = wilco_func(df_rsna[f'{typ}_TV_{angle}'], df_rsna[f'{typ}_U_Net_{angle}'])

    print(f"{angle}, FBP-TV: {stat_sparse_tv}, FBP-U-Net: {stat_sparse_U_Net}, TV-U-Net: {stat_TV_U_Net}")
    
typ="psnr"
for angle in angles:
    stat_sparse_tv = wilco_func(df_rsna[f'{typ}_sparse_{angle}'], df_rsna[f'{typ}_TV_{angle}'])
    stat_sparse_U_Net = wilco_func(df_rsna[f'{typ}_sparse_{angle}'], df_rsna[f'{typ}_U_Net_{angle}'])
    stat_TV_U_Net = wilco_func(df_rsna[f'{typ}_TV_{angle}'], df_rsna[f'{typ}_U_Net_{angle}'])

    print(f"{angle}, FBP-TV: {stat_sparse_tv}, FBP-U-Net: {stat_sparse_U_Net}, TV-U-Net: {stat_TV_U_Net}")

### CQ500 test split

In [ ]:
df_mean = pd.DataFrame()
df_cq = pd.read_csv("./ROI_psnr_ssim_CQ500.csv")
df_cq = df_cq.replace([np.inf, -np.inf], np.nan).dropna()
angles = [2048, 1024, 512, 256, 128, 64]

for angle in angles:
    column = []
    
    for typ in ["ssim", "psnr"]:
        for proc in ["sparse", "U_Net", "TV"]:

            print(angle, proc)
            metric_list = df_cq[f'{typ}_{proc}_{angle}']
            res = st.bootstrap((metric_list,), np.mean, n_resamples=1000, confidence_level=0.95)
            metric_interval = res.confidence_interval
            
            column.append(f'{round(metric_list.mean(), 3)}')
            column.append(f'({round(metric_interval[0], 3)}-{round(metric_interval[1], 3)})')
            
    df_mean[f"{angle}"] = column

            #check for normal distribution
            #print(f"{typ}", shapiro(metric_list))
df_mean.to_csv("./tableCQ500_ROI_SSIM_PSNR_raw.csv")

In [ ]:
#check if distributions are significantly different
angles = [2048, 1024, 512, 256, 128, 64]
typ="ssim"
for angle in angles:
    stat_sparse_tv = wilco_func(df_cq[f'{typ}_sparse_{angle}'], df_cq[f'{typ}_TV_{angle}'])
    stat_sparse_U_Net = wilco_func(df_cq[f'{typ}_sparse_{angle}'], df_cq[f'{typ}_U_Net_{angle}'])
    stat_TV_U_Net = wilco_func(df_cq[f'{typ}_TV_{angle}'], df_cq[f'{typ}_U_Net_{angle}'])

    print(f"{angle}, FBP-TV: {stat_sparse_tv}, FBP-U-Net: {stat_sparse_U_Net}, TV-U-Net: {stat_TV_U_Net}")
    
typ="psnr"
for angle in angles:
    stat_sparse_tv = wilco_func(df_cq[f'{typ}_sparse_{angle}'], df_cq[f'{typ}_TV_{angle}'])
    stat_sparse_U_Net = wilco_func(df_cq[f'{typ}_sparse_{angle}'], df_cq[f'{typ}_U_Net_{angle}'])
    stat_TV_U_Net = wilco_func(df_cq[f'{typ}_TV_{angle}'], df_cq[f'{typ}_U_Net_{angle}'])

    print(f"{angle}, FBP-TV: {stat_sparse_tv}, FBP-U-Net: {stat_sparse_U_Net}, TV-U-Net: {stat_TV_U_Net}")
